!["Facultad de Ingeniería de la UBA"](LogoFIUBA.jpg)


### Carrera de Maestría en Inteligencia Artificial (UBA) - _Cohorte 2_
## Análisis de Series Temporales II _(AST2)_
# **Trabajo Práctico 1:** _._

- #### Autor: Alejandro Lloveras _(a1716)_
- #### Docentes: 
    - Matias Vera
    - Camilo Argoty

- **Fecha de Entrega:** 16/11/2025

In [64]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import lognorm
from scipy.linalg import solve_discrete_lyapunov
from statsmodels.tsa.arima_process import ArmaProcess


---

## 1. Movimiento Browniano Geométrico (MBG)

El ejercicio debe ser entregado en un archivo `*.ipynb`.

El precio de una acción es actualmente **$P_0 = \$60$** por acción y sigue un movimiento browniano geométrico (MBG):

$$dP_{t}=\mu P_{t}dt+\sigma P_{t}dW_{t}$$

Asuma que el **retorno esperado $\mu$ es $20\%$ anual** y su **volatilidad $\sigma$ es $40\%$ anual**. 

1. Determine la distribución de probabilidad para el precio de la acción en **2 años ($P_2$)**.

2. Obtenga la **media** y la **desviación estándar** de dicha distribución y construya un **intervalo de confianza del $95\%$** para el precio de la acción.

3. Realice una **simulación de Montecarlo** y verifique si los cálculos anteriores coinciden con la simulación.

## Resolución

In [ ]:
# Constantes del problema
P0 = 60      # Precio inicial ($)
mu = 0.20    # Retorno esperado anual
sigma = 0.40 # Volatilidad anual
T = 2        # Tiempo en años (2 años)

print("DATOS PROVISTOS:")
print(f"- Precio Inicial (P0): ${P0:.2f}")
print(f"- Retorno Esperado (μ): {mu*100:.0f}%")
print(f"- Volatilidad (σ): {sigma*100:.0f}%")
print(f"- Horizonte (T): {T} años")

DATOS PROVISTOS:
- Precio Inicial (P0): $60.00
- Retorno Esperado (μ): 20%
- Volatilidad (σ): 40%
- Horizonte (T): 2 años


---

### 1.1 Distribución de Probabilidad

La solución a la SDE del MBG nos da el precio futuro $P_T$:

$$
P_T = P_0 \exp\left( \left(\mu - \frac{1}{2}\sigma^2\right)T + \sigma W_T \right)
$$

Donde $W_T$ es un proceso de Wiener en el tiempo $T$, que se distribuye como $W_T \sim N(0, T)$.

Para encontrar la distribución, analizamos su logaritmo:

$$
\ln(P_T) = \ln(P_0) + \left(\mu - \frac{1}{2}\sigma^2\right)T + \sigma W_T
$$

Dado que $W_T \sim N(0,T)$, la distribución del término $\sigma W_T \sim N(0, \sigma^2 T)$.

Por tratarse de una suma de constantes con una variable normal, el logaritmo del precio $\ln(P_T)$ seguirá una **distribución normal**:

$$
\ln(P_T) \sim N\left( \ln(P_0) + \left(\mu - \frac{1}{2}\sigma^2\right)T, \quad \sigma^2 T \right)
$$

Por lo tanto, el precio de la acción $P_T$ sigue una **distribución Log-Normal**.

In [39]:
# Primero, calculamos los parámetros de la distribución normal
# ln(P_T) ~ N(mu_log, sigma_log^2)

m = np.log(P0) + (mu - 0.5 * sigma**2) * T # Media de ln(P_T)
s = sigma * np.sqrt(T) # Desviación estándar de ln(P_T)

print("Distribución Log-Normal:")
print(f"  - Media (α): {m:.6f}")
print(f"  - Desv. Estándar (β): {s:.6f}")

Distribución Log-Normal:
  - Media (α): 4.334345
  - Desv. Estándar (β): 0.565685


### 1.2 Media, Desviación Estándar e Intervalo de Confianza

Para una variable log-normal $Y$ donde $\ln(Y) \sim N(\alpha, \beta^2)$, sus métricas de dispersión son:
* ***Media:*** $E[Y] = \exp(\alpha + \beta^2 / 2)$
* ***Varianza:*** $Var(Y) = [\exp(\beta^2) - 1] \exp(2\alpha + \beta^2)$

Para el caso particular de este problema:
- $Y = P_T$
- $\alpha = \ln(P_0) + (\mu - 0.5\sigma^2)T$
- $\beta^2 = \sigma^2 T$


#### Media $E[P_T]$

$$E[Y] = \exp(\alpha + \beta^2 / 2)$$

Sustituyendo $\alpha$ y $\beta^2$:
$$
E[P_T] = \exp\left( \left[\ln(P_0) + (\mu - 0.5\sigma^2)T\right] + (\sigma^2 T) / 2 \right)
$$
$$
E[P_T] = \exp(\ln(P_0) + \mu T)
$$
$$
E[P_T] = P_0 e^{\mu T}
$$


#### Varianza $Var(P_T)$

$$Var(Y) = [\exp(\beta^2) - 1] \exp(2\alpha + \beta^2)$$

Sustituyendo $\alpha$ y $\beta^2$:
$$
Var(P_T) = [\exp(\sigma^2 T) - 1] \exp\left(2[\ln(P_0) + (\mu - 0.5\sigma^2)T] + \sigma^2 T\right)
$$
$$
Var(P_T) = P_0^2 e^{2\mu T} (\exp(\sigma^2 T) - 1)
$$


#### Desviación Estándar $Std(P_T)$

Es la raíz cuadrada de la varianza:
$$
Std(P_T) = \sqrt{Var(P_T)} = P_0 e^{\mu T} \sqrt{\exp(\sigma^2 T) - 1}
$$

In [41]:
media_teorica = P0 * np.exp(mu * T) # Media E[P_T]
std_teorica = media_teorica * np.sqrt(np.exp(sigma**2 * T) - 1) # Desviación Estándar Std(P_T)

print("\nDistribución del Precio P_T:")
print(f"- Media Teórica E[P_T]: ${media_teorica:.2f}")
print(f"- Desv. Estándar Teórica: ${std_teorica:.2f}")


Distribución del Precio P_T:
- Media Teórica E[P_T]: $89.51
- Desv. Estándar Teórica: $54.97


#### Intervalo de Confianza del 95%
Construimos el intervalo sobre $\ln(P_T)$ y luego exponenciamos los límites.


El IC del 95% para $\ln(P_T)$ es:
$$[\alpha - 1.96 \cdot \beta, \quad \alpha + 1.96 \cdot \beta]$$

Donde $\alpha = \ln(P_T)_{\text{mean}}$ y $\beta = \ln(P_T)_{\text{std}} = \sigma \sqrt{T}$.

El IC del 95% para $P_T$ es:
$$[ \exp(\alpha - 1.96 \beta), \quad \exp(\alpha + 1.96 \beta) ]$$

In [48]:
# Utilizando Scipy para obtener el intervalo de confianza
dist_teorica = lognorm(s=s, scale=np.exp(m)) # Distribución Lognormal
ic_teorico = dist_teorica.interval(0.95) # Intervalo de Confianza 95%

print(f"Intervalo Confianza 95%:")
print(f"- Extremo inferior: ${ic_teorico[0]:.2f}")
print(f"- Extremo superior: ${ic_teorico[1]:.2f}")

Intervalo Confianza 95%:
- Extremo inferior: $25.17
- Extremo superior: $231.15


### 1.3 Simulación de Montecarlo

In [50]:
N_SIMS = 1_000_000 # Número de simulaciones

# 1. Generamos N_SIMS números aleatorios de N(0, 1)
Z = np.random.normal(0, 1, N_SIMS)

# 2. Calculamos el precio final P_T para cada simulación
# P_T = P0 * exp( (mu - 0.5*sigma^2)*T + sigma*sqrt(T)*Z )
drift = (mu - 0.5 * sigma**2) * T
diffusion = sigma * np.sqrt(T)
P_T_simulados = P0 * np.exp(drift + diffusion * Z)

# 3. Estadísticas de los resultados simulados
media_simulada = np.mean(P_T_simulados)
std_simulada = np.std(P_T_simulados)
ic_simulado = np.percentile(P_T_simulados, [2.5, 97.5])

print("SIMULACIÓN DE MONTECARLO")
print(f"- Simulaciones realizadas: {N_SIMS:,}")
print(f"- Media Simulada:           ${media_simulada:.2f}")
print(f"- Desv. Estándar Simulada:  ${std_simulada:.2f}")
print(f"- Intervalo Confianza 95%:  [${ic_simulado[0]:.2f}, ${ic_simulado[1]:.2f}]")


SIMULACIÓN DE MONTECARLO
- Simulaciones realizadas: 1,000,000
- Media Simulada:           $89.46
- Desv. Estándar Simulada:  $54.90
- Intervalo Confianza 95%:  [$25.22, $231.28]


### Conclusión
Se verifican con la simulación de Montecarlo los valores calculados en el desarrollo teórico.

In [56]:
print("|-----------------------|---------------|---------------|")
print("| Métrica               | Teórico       | Simulado      |")
print("|-----------------------|---------------|---------------|")
print(f"| Media                 | ${media_teorica: <12.2f} | ${media_simulada: <12.2f} |")
print(f"| Desviación Estándar   | ${std_teorica: <12.2f} | ${std_simulada: <12.2f} |")
print(f"| IC 95% (Inferior)     | ${ic_teorico[0]: <12.2f} | ${ic_simulado[0]: <12.2f} |")
print(f"| IC 95% (Superior)     | ${ic_teorico[1]: <12.2f} | ${ic_simulado[1]: <12.2f} |")
print("|-----------------------|---------------|---------------|")

|-----------------------|---------------|---------------|
| Métrica               | Teórico       | Simulado      |
|-----------------------|---------------|---------------|
| Media                 | $89.51        | $89.46        |
| Desviación Estándar   | $54.97        | $54.90        |
| IC 95% (Inferior)     | $25.17        | $25.22        |
| IC 95% (Superior)     | $231.15       | $231.28       |
|-----------------------|---------------|---------------|



---

## 2. Proceso Estocástico ARMA(5,3) y Filtro de Kalman

El ejercicio debe ser entregado en un archivo `*.ipynb`.
Considere un proceso estocástico discreto que sigue un **modelo ARMA (5,3)**.

1. Genere una **secuencia temporal sintética** de dicho proceso de largo **$T=10000$**.

2. Suponga que las observaciones de la señal están disponibles únicamente cada **10 instantes de tiempo** (submuestreo).

3. Formule e implemente un **filtro de Kalman** que permita **estimar los estados intermedios** y realizar **pronósticos**.

4. Evalúe el desempeño del filtro comparando las estimaciones con los valores verdaderos de la señal simulada. 